## Import

In [ ]:
import pandas as pd
import numpy as np

import torch
from torch import nn, optim
import torch.nn as nn
import torch.optim as optim
from torch.nn.parameter import Parameter
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer

ImportError: tokenizers>=0.14,<0.19 is required for a normal functioning of this module, but found tokenizers==0.20.3.
Try: `pip install transformers -U` or `pip install -e '.[dev]'` if you're working with git main

## Pretrained Model 불러오기
- 둘 중 하나 쓰면 됨(tokenizer&pretrained_model 세트로)

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('j5ng/et5-typos-corrector')
pretrained_model = T5ForConditionalGeneration.from_pretrained('j5ng/et5-typos-corrector')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("cosmoquester/bart-ko-mini")
pretrained_model = AutoModelForSeq2SeqLM.from_pretrained("cosmoquester/bart-ko-mini")

## Data Load

In [ ]:
train = pd.read_csv("강석호교수님.csv")
test = pd.read_csv("강명인교수님.csv")

In [ ]:
# train 데이터의 라벨(실제 교수님께서 하신 말씀)을 띄어쓰기 단위(단어 단위라고 가정)로 나눠서 기존 token(logit을 가질 label)에 추가
add_tokens = [word for sentence in train['y'] for word in sentence.split()]
tokenizer.add_tokens(add_tokens)

# token이 추가되었으니(출력되어야할 차원이 커짐) embedding 사이즈 수정 
pretrained_model.resize_token_embeddings(len(tokenizer))

Embedding(34138, 256)

In [ ]:
# 문장을 단어 단위로 나누고 tokenizer -> 추후 사용에 용이하도록 dictionary 형태로 만듬
def tokenizer_(input):
    input_list = [input for input in input['X']]
    label_list = [label for label in input['y']]

    input_tokenizer = tokenizer(input_list, return_tensors="pt", max_length=32, truncation=True, padding="max_length")['input_ids']
    label_tokenizer = tokenizer(label_list, return_tensors="pt", max_length=32, truncation=True, padding="max_length")['input_ids']

    tokenizer_dict = {'input_tokenizer': input_tokenizer, 'label_tokenizer': label_tokenizer}
    return tokenizer_dict

In [ ]:
train_1 = train.iloc[:476,:] # 사실 이거는 하면 안되긴 함 -> 학습 시간이 너무 오래걸려서 augmentation 했던거 잘라서 학습 시도해본 것
dataset = tokenizer_(train_1) # train 데이터 토큰화

In [ ]:
# test 데이터 토큰화
Inference_dataset = tokenizer_(test)

## 모델 구현

In [ ]:
class KoBART(nn.Module):
    def __init__(self, pretrained_model):
        super(KoBART, self).__init__()

        self.pretrained_model = pretrained_model
        self.output_size = 34138 # token 사이즈
        self.hidden_size = 256

        # 위에 우리가 쌓고자 했던 layer
        self.new_fc = nn.Sequential(
            nn.Linear(self.hidden_size, self.hidden_size),
            nn.ReLU(),
            nn.Linear(self.hidden_size, self.hidden_size),
            nn.ReLU(),
            nn.Linear(self.hidden_size, self.hidden_size),
            nn.ReLU(),
            nn.Linear(self.hidden_size, self.output_size)
        )                  

    def forward(self, input):
        input = torch.reshape(input, (1,-1))
        new_input = self.pretrained_model(input, output_hidden_states=True).decoder_hidden_states[-1] # pretrained model 학습 후 마지막 layer logit만 불러옴
        new_output = self.new_fc(new_input) # 새로 쌓은 layer에 넣음음
        return new_output
        

## 학습

In [684]:
device = 'cpu'
model = KoBART(pretrained_model)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.new_fc.parameters(), lr=0.01)
epochs = 10

In [ ]:
model.train()
for epoch in tqdm(range(epochs)):
    temp_loss = 0

    # mini batch
    for idx in tqdm(range(len(train_1))):
        logit = model(dataset['input_tokenizer'][idx]) # dataset['input_tokenizer'] = input train 토큰하된 것
        pred = torch.reshape(logit, (32,34138))
        loss = loss_fn(pred, dataset['label_tokenizer'][idx])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        temp_loss += loss
    
    print(f"{epoch} loss: {temp_loss.detach().item()}")

 10%|█         | 1/10 [01:24<12:39, 84.35s/it]

0 loss: 1336.108642578125


 20%|██        | 2/10 [02:36<10:15, 76.89s/it]

1 loss: 1093.953369140625


 30%|███       | 3/10 [04:11<09:58, 85.53s/it]

...

7 loss: 841.7211303710938


 90%|█████████ | 9/10 [13:45<01:43, 103.21s/it]

8 loss: 825.9454956054688


100%|██████████| 10/10 [15:38<00:00, 93.86s/it] 

9 loss: 812.2211303710938


## Inference

In [ ]:
# 여기가 위에 학습한 모델로 추론한 것(근데 학습 시간이 너무 오래걸려서 끝까지 학습 안 함함)
model.eval()
model.to('cpu')

result = []
for sentence in tqdm(Inference_dataset['input_tokenizer']):
    input = torch.reshape(sentence, (1,-1))
    logit = model(input)
    output = torch.argmax(logit[0], dim=1)
    correction = tokenizer.decode(output, skip_special_tokens=True)
    result.append(correction)

100%|██████████| 648/648 [00:27<00:00, 23.15it/s]


In [ ]:
correction_df = pd.DataFrame(result, columns=['y_hat'])
total_df = pd.concat([correction_df, test['y']], axis=1)

In [688]:
total_df

,y_hat,y
0,우리가,안녕하세요 데이터과학을 위한 프로그래밍 수강생 여러분
1,우리가,오늘 슈퍼바이스러닝 다섯번째 시간 시작하도록 하겠습니다
2,그래서,오늘 할 내용은 뉴럴 네트워크 이구요
3,그래서 게,원래 서포트 벡터 머신이 먼저 나와 있는데
4,그래서 수 수,"순서를 바꿔서 인공신경망, 뉴로이 네트워크에 관련된 내용부터 진행하도록 하겠습니다"
...,...,...
643,우리가,조금 더 빠르게 수렴할 수 수렴시킬 수 있고
644,우리가,그리고 성능도 좋을 수 있다 라는 일반적인 이야기가 있고요
645,우리가 게,그런데 이제 사실 만약에 사이킷런 말고
646,우리가 게,여러분이 뭔가 좀 더 텐서플로우나


In [ ]:
# 여기는 결과가 너무 안 좋아서 그냥 pretrained 모델만 갖고(resize된 상태) decode 시킴(추가 layer을 안 쌓은 것)
result1 = []
for sentence in tqdm(Inference_dataset['input_tokenizer']):
    correction = tokenizer.decode(model.pretrained_model.generate(torch.reshape(sentence,(1,-1)),max_length=50)[0], skip_special_tokens=True)
    result1.append(correction)

100%|██████████| 648/648 [01:53<00:00,  5.69it/s]


In [655]:
correction_df1 = pd.DataFrame(result1, columns=['y_hat'])
total_df1 = pd.concat([correction_df1, test['y']], axis=1)

In [656]:
total_df1

,y_hat,y
0,[BOS]녕하세요 [UNK]과학을 [EOS],안녕하세요 데이터과학을 위한 프로그래밍 수강생 여러분
1,[BOS] 관공서 다섯 한 시간 [EOS],오늘 슈퍼바이스러닝 다섯번째 시간 시작하도록 하겠습니다
2,[BOS] 할 뉴로이 네트워크 이구요[EOS],오늘 할 내용은 뉴럴 네트워크 이구요
3,[BOS] [EOS],원래 서포트 벡터 머신이 먼저 나와 있는데
4,[BOS]서를 바꿔서 뉴로이 [EOS],"순서를 바꿔서 인공신경망, 뉴로이 네트워크에 관련된 내용부터 진행하도록 하겠습니다"
...,...,...
643,[BOS] 더 [UNK] 수렴할 수 수 수렴시킬[EOS],조금 더 빠르게 수렴할 수 수렴시킬 수 있고
644,[BOS] [UNK] 라는 [EOS],그리고 성능도 좋을 수 있다 라는 일반적인 이야기가 있고요
645,[BOS] 사이클론 말고[EOS],그런데 이제 사실 만약에 사이킷런 말고
646,[BOS]이 뭔가 좀 더 텐서플로우나[EOS],여러분이 뭔가 좀 더 텐서플로우나
